In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import * 
import urllib
import os

In [ ]:
#AWS cred locations
dbutils.fs.ls("/FileStore/tables")

Out[793]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-12853887c065-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount_name"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
#dbutils are not supported outside of databricks notebook
# dbutils.fs.mount(SOURCE_URL, MOUNT_NAME) # RUN ONCE ONLY!

display(dbutils.fs.ls("/mnt/mount_name/topics/12853887c065.pin/partition=0/"))

# dbutils.fs.unmount("/mnt/mount_name") #unmount if it is already mounted

In [ ]:
# # Adding the packages required to get data from S3  
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.amazonaws:aws-java-sdk-s3:1.12.490,org.apache.hadoop:hadoop-aws:3.3.1 pyspark-shell"

# File location and type
pin_location = "/mnt/mount_name/topics/12853887c065.pin/partition=0/*.json"
geo_location = "/mnt/mount_name/topics/12853887c065.geo/partition=0/*.json"
user_location = "/mnt/mount_name/topics/12853887c065.user/partition=0/*.json" 
file_type = "json"

# Ask Spark to infer the schema
infer_schema = "true"

# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(pin_location)
df_geo = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(geo_location)
df_user = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(user_location)

df_pin.show(5, True)
df_geo.show(5, True)
df_user.show(5, True)

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
 tattoos|Image uploaded by...| 1| 15M|https://i.pinimg....| 8586| image| We Heart It|Local save in /da...|Mens Body Tattoos...| Pretty ♥ discover...|c338b1c8-7c6a-4a1...|
event-planning|15.1k Likes, 83 C...| 1| 311|https://i.pinimg....| 4858| image| Marie Bradford|Local save in /da...|Diy Birthday Deco...| THE EVENT COLLECT...|58101415-9273-431...|
event-planning|Wow your guests! ...| 1| 1k|https://i.pinimg....| 4608| image|Iconica Design | ...|Local save in /da...|Christmas Party B...| Virtual Baby Show...|d234e56f-5b18-4ef...|
 home-decor|6,636 Likes, 141 ...| 1| 394|https://i.pinimg....| 6633| image| Sarah Martin|Local save in /da...|Diy Kitchen Cabin...| The Cottage Journ...|d136f6bc-840d-44f...|
 home-decor|И хоть у шведов л...| 1| 136k|https://i.pinimg....| 6447| image|PUFIK Interiors &...|Local save in /da...|Cheap Home Decor,...|〚 Warm natural to...|d3039535-5767-426...|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
only showing top 5 rows

+--------------------+----+--------+---------+-------------------+
 country| ind|latitude|longitude| timestamp|
+--------------------+----+--------+---------+-------------------+
British Indian Oc...|9314|-42.0773| -163.698|2020-10-20 07:10:24|
British Indian Oc...|9455|-82.9272| -150.346|2022-03-15 01:46:32|
British Indian Oc...|7088|-86.5675| -149.565|2020-10-16 07:16:02|
British Indian Oc...|6945|-14.6744| -75.3714|2022-09-13 22:37:32|
British Indian Oc...|5111|-83.7472| 8.65953|2021-04-01 00:56:57|
+--------------------+----+--------+---------+-------------------+
only showing top 5 rows

+---+-------------------+-----------+----+---------+
age| date_joined| first_name| ind|last_name|
+---+-------------------+-----------+----+---------+
 38|2015-11-12 11:07:24|Christopher|3151|Hernandez|
 25|2016-11-24 18:36:05|Christopher|3587|Rodriguez|
 27|2016-03-08 13:38:37|Christopher|2015| Bradshaw|
 27|2016-03-08 13:38:37|Christopher|2015| Bradshaw|
 48|2016-02-27 16:57:44|Christopher|1857| Hamilton|
+---+-------------------+-----------+----+---------+
only showing top 5 rows

In [ ]:

#Check for null values
# df_pin.filter(f.greatest(*[f.col(i).isNull() for i in df_pin.columns])).show()

# Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
df_pin = df_pin.withColumn("follower_count", \
    when((df_pin.follower_count.contains('k')),\
        1000*regexp_replace(df_pin["follower_count"], 'k', '').cast('int'))
    .when((df_pin.follower_count.contains('M')),\
        1000000*regexp_replace(df_pin['follower_count'], 'M', '').cast('int'))
    .otherwise(df_pin.follower_count).cast('int'))

#Cast Data types on columns
df_pin = df_pin.withColumn("downloaded",col("downloaded").cast("int"))
df_pin = df_pin.withColumn("index",col("index").cast("int"))

# Clean the data in the save_location column to include only the save location path
df_pin = df_pin.withColumn('save_location', split(df_pin.save_location, ' ').getItem(3))

# Rename the index column to ind.
df_pin = df_pin.withColumnRenamed("index","ind")

# Reorder the DataFrame columns to have the following column order:
new_column = ['ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category']
df_pin = df_pin.select(new_column)

df_pin.select('ind','follower_count', 'save_location', ).show(10)

+----+--------------+--------------------+
 ind|follower_count| save_location|
+----+--------------+--------------------+
8586| 15000000| /data/tattoos|
4858| 311|/data/event-planning|
4608| 1000|/data/event-planning|
6633| 394| /data/home-decor|
6447| 136000| /data/home-decor|
1706| 5000| /data/christmas|
8161| 19000| /data/quotes|
9989| 6000| /data/travel|
1777| 5000| /data/christmas|
1676| 784| /data/christmas|
+----+--------------+--------------------+
only showing top 10 rows

In [ ]:
#Create a new column coordinates that contains an array based on the latitude and longitude columns
df_geo = df_geo.withColumn('coordinates', array('latitude', 'longitude'))

#Drop the latitude and longitude columns from the DataFrame
df_geo = df_geo.drop('latitude', 'longitude')

#Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn('timestamp', col('timestamp').cast('timestamp'))

#Reorder the DataFrame columns to have the following column order:
col_order = ['ind', 'country', 'coordinates', 'timestamp']
df_geo = df_geo.select(col_order)

df_geo.show(5)

+----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+----+--------------------+--------------------+-------------------+
9314|British Indian Oc...|[-42.0773, -163.698]|2020-10-20 07:10:24|
9455|British Indian Oc...|[-82.9272, -150.346]|2022-03-15 01:46:32|
7088|British Indian Oc...|[-86.5675, -149.565]|2020-10-16 07:16:02|
6945|British Indian Oc...|[-14.6744, -75.3714]|2022-09-13 22:37:32|
5111|British Indian Oc...| [-83.7472, 8.65953]|2021-04-01 00:56:57|
+----+--------------------+--------------------+-------------------+
only showing top 5 rows

In [ ]:
#Create a new column user_name that concatenates the information found in the first_name and last_name columns
df_user = df_user.withColumn('user_name', concat_ws(' ', 'first_name', 'last_name'))

#Drop the first_name and last_name columns from the DataFrame
df_user = df_user.drop('first_name', 'last_name')

#Convert the date_joined column from a string to a timestamp data type
df_user = df_user.withColumn('date_joined', col('date_joined').cast('timestamp'))

# Reorder the DataFrame columns to have the following column order:
user_col = ['ind', 'user_name', 'age', 'date_joined']
df_user = df_user.select(user_col)
df_user.show(5)

+----+--------------------+---+-------------------+
 ind| user_name|age| date_joined|
+----+--------------------+---+-------------------+
3151|Christopher Herna...| 38|2015-11-12 11:07:24|
3587|Christopher Rodri...| 25|2016-11-24 18:36:05|
2015|Christopher Bradshaw| 27|2016-03-08 13:38:37|
2015|Christopher Bradshaw| 27|2016-03-08 13:38:37|
1857|Christopher Hamilton| 48|2016-02-27 16:57:44|
+----+--------------------+---+-------------------+
only showing top 5 rows

In [ ]:
# Find the most popular Pinterest category people post to based on their country.
# First join the dataframe
df_pin_geo = df_pin.join(df_geo, df_pin.ind==df_geo.ind, 'inner')

#Groupby country and category
df_pin_geo.groupby('country','category') \
    .agg(count('category')\
        .alias('category_count'))\
    .sort('category_count', ascending=False)\
    .show(10)

# df_pin_geo.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Albania| mens-fashion| 151|
 Armenia|diy-and-crafts| 81|
 Colombia| finance| 66|
 Algeria| quotes| 51|
 Albania| art| 50|
Cocos (Keeling) I...| vehicles| 36|
 French Guiana| quotes| 36|
 Anguilla| home-decor| 36|
 Albania|diy-and-crafts| 34|
 Azerbaijan| finance| 33|
+--------------------+--------------+--------------+
only showing top 10 rows

In [ ]:
# Find how many posts each category had between 2018 and 2022.

#Create temporary view 
df_pin_geo.createOrReplaceTempView("pin_geo")

# Use spark sql queries
spark.sql(\
    'SELECT\
        YEAR(timestamp) as post_year,\
        category,\
        count(category) category_count\
    FROM pin_geo\
    WHERE YEAR(timestamp) BETWEEN 2018 AND 2022\
    GROUP BY post_year, category\
    ORDER BY post_year, category_count DESC')\
    .show(10)

+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| beauty| 84|
 2018|diy-and-crafts| 76|
 2018| mens-fashion| 65|
 2018| travel| 61|
 2018| quotes| 54|
 2018| education| 54|
 2018| christmas| 51|
 2018|event-planning| 46|
 2018| art| 38|
 2018| tattoos| 35|
+---------+--------------+--------------+
only showing top 10 rows

In [ ]:
# Find how many posts each category had between 2018 and 2022.

windowSpec = Window.partitionBy(df_pin_geo.category)\
                    .orderBy(df_pin_geo.timestamp)

df_pin_geo.filter((year('timestamp') >= 2018) & (year('timestamp') <= 2022))\
    .select(
        year('timestamp') \
        .alias('post_year'), 'category', count('category') \
        .over(windowSpec) \
        .alias('category_count')) \
    .show(10)

+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| art| 1|
 2019| art| 2|
 2019| art| 3|
 2021| art| 4|
 2021| art| 5|
 2021| art| 6|
 2022| art| 7|
 2018| beauty| 1|
 2018| beauty| 2|
 2018| beauty| 3|
 2018| beauty| 4|
 2018| beauty| 5|
 2019| beauty| 6|
 2020| beauty| 7|
 2022| beauty| 8|
 2018| christmas| 1|
 2019| christmas| 2|
 2019| christmas| 3|
 2019| christmas| 4|
 2019| christmas| 5|
 2019| christmas| 6|
 2020| christmas| 7|
 2020| christmas| 8|
 2021| christmas| 9|
 2022| christmas| 10|
 2022| christmas| 11|
 2018|diy-and-crafts| 1|
 2018|diy-and-crafts| 2|
 2019|diy-and-crafts| 3|
 2019|diy-and-crafts| 4|
 2020|diy-and-crafts| 5|
 2020|diy-and-crafts| 9|
 2020|diy-and-crafts| 9|
 2020|diy-and-crafts| 9|
 2020|diy-and-crafts| 9|
 2020|diy-and-crafts| 10|
 2021|diy-and-crafts| 11|
 2021|diy-and-crafts| 12|
 2021|diy-and-crafts| 13|
 2022|diy-and-crafts| 14|
 2022|diy-and-crafts| 15|
 2022|diy-and-crafts| 16|
 2018| education| 1|
 2018| education| 2|
 2019| education| 3|
 2019| education| 4|
 2019| education| 5|
 2019| education| 6|
 2019| education| 7|
 2020| education| 8|
 2019|event-planning| 1|
 2021|event-planning| 2|
 2021|event-planning| 3|
 2021|event-planning| 4|
 2021|event-planning| 5|
 2022|event-planning| 6|
 2022|event-planning| 7|
 2019| finance| 1|
 2019| finance| 2|
 2020| finance| 3|
 2020| finance| 4|
 2021| finance| 8|
 2021| finance| 8|
 2021| finance| 8|
 2021| finance| 8|
 2021| finance| 9|
 2021| finance| 10|
 2021| finance| 11|
 2022| finance| 12|
 2018| home-decor| 1|
 2019| home-decor| 2|
 2019| home-decor| 3|
 2020| home-decor| 4|
 2020| home-decor| 5|
 2020| home-decor| 6|
 2021| home-decor| 7|
 2021| home-decor| 8|
 2018| mens-fashion| 1|
 2018| mens-fashion| 2|
 2018| mens-fashion| 3|
 2019| mens-fashion| 4|
 2020| mens-fashion| 5|
 2020| mens-fashion| 6|
 2020| mens-fashion| 10|
 2020| mens-fashion| 10|
 2020| mens-fashion| 10|
 2020| mens-fashion| 10|
 2021| mens-fashion| 11|
 2021| mens-fashion| 12|
 2021| mens-fashion| 13|
 2022| mens-fashion| 14|
 2022| mens-fashion| 15|
 2018| quotes| 1|
 2019| quotes| 5|
 2019| quotes| 5|
 2019| quotes| 5|
 2019| quotes| 5|
 2019| tattoos| 1|
 2020| tattoos| 2|
 2020| tattoos| 3|
+---------+--------------+--------------+
only showing top 100 rows

In [ ]:
#Find the user with most followers

# Step 1: For each country find the user with the most followers.
spark.sql("WITH most_follower_table AS (\
                SELECT pin_geo.country,\
                    pin_geo.poster_name,\
                    SUM(pin_geo.follower_count) total_follower\
                FROM pin_geo\
                GROUP BY pin_geo.country, pin_geo.poster_name\
        )\
    SELECT country,\
        sum(total_follower) follower_count\
    FROM most_follower_table\
    GROUP BY country\
    ORDER BY follower_count DESC\
    LIMIT 1").show()

+-------+--------------+
country|follower_count|
+-------+--------------+
Armenia| 1866000|
+-------+--------------+

In [ ]:
df_pin_geo.select(df_pin_geo.country, df_pin_geo.poster_name, df_pin_geo.follower_count)\
            .groupby(df_pin_geo.country)\
            .agg(sum(df_pin_geo.follower_count).alias('follower_count'))\
            .orderBy(col('follower_count').desc()).show(1)

+-------+--------------+
country|follower_count|
+-------+--------------+
Armenia| 1866000|
+-------+--------------+
only showing top 1 row

In [ ]:
# What is the most popular category people post to based on the following age groups: 18-24, 25-35, 36-50, +50
#Join df_pin and df_user
df_pin_user = df_pin.join(df_user, df_pin.ind==df_user.ind, 'inner')

#Create temporary view 
df_pin_user.createOrReplaceTempView("pin_user")
spark.sql("SELECT\
    CASE\
        WHEN age BETWEEN 18 AND 24 THEN '18-24' \
        WHEN age BETWEEN 25 AND 35 THEN '25-35'\
        WHEN age BETWEEN 36 AND 50 THEN '36-50'\
        WHEN age > 50 THEN '50+'\
    END AS age_group,\
    category,\
    COUNT(category) AS category_count\
    FROM pin_user\
    GROUP BY age_group, category\
    ORDER BY age_group ASC, category_count DESC").show(10)

+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| mens-fashion| 11|
 18-24| christmas| 7|
 18-24|diy-and-crafts| 5|
 18-24|event-planning| 5|
 18-24| home-decor| 5|
 18-24| travel| 4|
 18-24| art| 4|
 18-24| tattoos| 4|
 18-24| education| 3|
 18-24| finance| 3|
 18-24| beauty| 3|
 18-24| quotes| 1|
 18-24| vehicles| 1|
 25-35|diy-and-crafts| 8|
 25-35| travel| 4|
 25-35| quotes| 4|
 25-35| art| 3|
 25-35| education| 3|
 25-35| finance| 3|
 25-35| mens-fashion| 3|
+---------+--------------+--------------+
only showing top 20 rows

In [ ]:
spark.sql("\
        WITH category_table AS (\
            SELECT\
                CASE\
                    WHEN age BETWEEN 18 AND 24 THEN '18-24' \
                    WHEN age BETWEEN 25 AND 35 THEN '25-35'\
                    WHEN age BETWEEN 36 AND 50 THEN '36-50'\
                    WHEN age > 50 THEN '50+'\
                END AS age_group,\
                category,\
                COUNT(category) category_count\
            FROM pin_user\
            GROUP BY age_group, category\
            )\
        SELECT\
            age_group,\
            category,\
            MAX(category_count)\
        FROM category_table\
        GROUP BY age_group, category"
    ).show(10)

+---------+--------------+-------------------+
age_group| category|max(category_count)|
+---------+--------------+-------------------+
 25-35|event-planning| 1|
 18-24| travel| 4|
 36-50| education| 1|
 25-35| art| 3|
 25-35| finance| 3|
 18-24| education| 3|
 50+| finance| 2|
 18-24|diy-and-crafts| 5|
 18-24|event-planning| 5|
 25-35| education| 3|
 36-50| christmas| 3|
 25-35|diy-and-crafts| 8|
 50+| home-decor| 1|
 36-50|diy-and-crafts| 3|
 50+| beauty| 2|
 25-35| travel| 4|
 18-24| christmas| 7|
 18-24| mens-fashion| 11|
 18-24| finance| 3|
 36-50| home-decor| 2|
+---------+--------------+-------------------+
only showing top 20 rows

In [ ]:
# What is the most popular category people post to based on the following age groups: 18-24, 25-35, 36-50, +50

#Add age-group col
df_pin_user = df_pin_user.withColumn('age_group',\
    when((df_pin_user.age >= 18) & (df_pin_user.age <=25), '18-25')
    .when((df_pin_user.age >= 26) & (df_pin_user.age <= 35), '26-35')
    .when((df_pin_user.age >= 36) & (df_pin_user.age <= 50), '36-50')
    .otherwise('50+'))

#Window specification
pin_user_windowSpec = Window.partitionBy('age_group')\
    .orderBy('category')



df_pin_user.select('age_group',\
    'category',\
    count(col('category'))\
    .over(pin_user_windowSpec)\
    .alias('category_count'))\
    .show()


+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-25| art| 4|
 18-25| art| 4|
 18-25| art| 4|
 18-25| art| 4|
 18-25| beauty| 7|
 18-25| beauty| 7|
 18-25| beauty| 7|
 18-25| christmas| 14|
 18-25| christmas| 14|
 18-25| christmas| 14|
 18-25| christmas| 14|
 18-25| christmas| 14|
 18-25| christmas| 14|
 18-25| christmas| 14|
 18-25|diy-and-crafts| 19|
 18-25|diy-and-crafts| 19|
 18-25|diy-and-crafts| 19|
 18-25|diy-and-crafts| 19|
 18-25|diy-and-crafts| 19|
 18-25| education| 22|
+---------+--------------+--------------+
only showing top 20 rows

In [ ]:
# What is the median follower count for users in the following age groups: 18-24, 25-35, 36-50, +50

spark.sql("\
    SELECT\
        CASE\
            WHEN age BETWEEN 18 AND 24 THEN '18-24' \
            WHEN age BETWEEN 25 AND 35 THEN '25-35'\
            WHEN age BETWEEN 36 AND 50 THEN '36-50'\
            WHEN age > 50 THEN '50+'\
        END AS age_group,\
        round(mean(follower_count), 3) mean_follower_count,\
        max(follower_count) min_follower_count,\
        min(follower_count) min_follower_count\
    FROM pin_user\
    GROUP BY age_group\
    ORDER BY age_group\
").show()


+---------+-------------------+------------------+------------------+
age_group|mean_follower_count|min_follower_count|min_follower_count|
+---------+-------------------+------------------+------------------+
 18-24| 256477.273| 985000| 2000|
 25-35| 168757.576| 972000| 1000|
 36-50| 42055.556| 314000| 0|
 50+| 121500.0| 637000| 2000|
+---------+-------------------+------------------+------------------+

In [ ]:
df_pin_user.groupby('age_group')\
    .agg(sum('follower_count').alias('total_follower_count'),\
        round(mean('follower_count'), 3).alias('mean_follower_count'))\
    .show()

+---------+--------------------+-------------------+
age_group|total_follower_count|mean_follower_count|
+---------+--------------------+-------------------+
 18-25| 11506000| 234816.327|
 26-35| 5348000| 191000.0|
 50+| 972000| 121500.0|
 36-50| 757000| 42055.556|
+---------+--------------------+-------------------+

In [ ]:
#Find how many users have joined between 2015 and 2020.

# Join user and geo df
df_geo_user = df_geo.join(df_user, df_geo.ind==df_user.ind, 'inner')

# df_geo_user.withColumn('timestamp', YEAR(col('timestamp'))).show()

df_geo_user.groupby(year('timestamp').alias('post_year'))\
    .agg(count(year('timestamp')).alias('number_users_joined'))\
    .sort('number_users_joined')\
    .show()

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2017| 1|
 2018| 17|
 2022| 18|
 2021| 24|
 2020| 27|
 2019| 30|
+---------+-------------------+

In [ ]:
# Find the median follower count of users have joined between 2015 and 2020.
df_pin_geo.filter((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
    .groupby(year('timestamp').alias('post_year'))\
    .agg(round(mean('follower_count') ,2).alias('median_follower_count'))\
    .orderBy('post_year')\
    .show() #median function not defined. Testing with mean

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2017| null|
 2018| 220307.69|
 2019| 152107.14|
 2020| 182000.0|
+---------+---------------------+

In [ ]:
#Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
# ~Join pin, geo & user df
df_pin_geo_user = df_pin.join(df_geo, df_pin.ind==df_geo.ind, 'inner')\
    .join(df_user, df_geo.ind==df_user.ind, 'inner')

#Add age-group col
df_pin_geo_user = df_pin_geo_user.withColumn('age_group',\
    when((df_pin_geo_user.age >= 18) & (df_pin_geo_user.age <=25), '18-25')
    .when((df_pin_geo_user.age >= 26) & (df_pin_geo_user.age <= 35), '26-35')
    .when((df_pin_geo_user.age >= 36) & (df_pin_geo_user.age <= 50), '36-50')
    .otherwise('50+'))


df_pin_geo_user.filter((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
    .groupby('age_group',year('timestamp')\
    .alias('post_year')).agg(mean('follower_count')
    .alias('median_follower_count'))
    .show()

+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 18-25| 2017| null|
 50+| 2019| 29333.333333333332|
 36-50| 2019| 43833.333333333336|
 18-25| 2018| 176500.0|
 26-35| 2019| 57600.0|
 18-25| 2019| 212444.44444444444|
 26-35| 2018| 194666.66666666666|
 50+| 2018| 637000.0|
 36-50| 2020| 97500.0|
 18-25| 2020| 276727.2727272727|
 26-35| 2020| 104600.0|
 50+| 2020| 20000.0|
+---------+---------+---------------------+

In [ ]:
windowSpec = Window.partitionBy('age_group')\
                    .orderBy('timestamp')

# median function not imported, testing with mean
df_pin_geo_user.filter((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
    .select('age_group', year('timestamp')
    .alias('post_year'), mean('follower_count')
    .over(windowSpec).alias('mean_follower_count'))
    .show()

+---------+---------+-------------------+
age_group|post_year|mean_follower_count|
+---------+---------+-------------------+
 18-25| 2017| null|
 18-25| 2018| 11000.0|
 18-25| 2018| 19500.0|
 18-25| 2018| 16666.666666666668|
 18-25| 2018| 16666.666666666668|
 18-25| 2018| 16666.666666666668|
 18-25| 2018| 16666.666666666668|
 18-25| 2018| 16666.666666666668|
 18-25| 2018| 14500.0|
 18-25| 2018| 208600.0|
 18-25| 2018| 176500.0|
 18-25| 2019| 152000.0|
 18-25| 2019| 135500.0|
 18-25| 2019| 135500.0|
 18-25| 2019| 224888.88888888888|
 18-25| 2019| 202900.0|
 18-25| 2019| 186454.54545454544|
 18-25| 2019| 185166.66666666666|
 18-25| 2019| 238076.92307692306|
 18-25| 2019| 221785.7142857143|
+---------+---------+-------------------+
only showing top 20 rows